In [1]:
import pandas as pd
import pickle
import joblib
# df = pd.read_pickle('poses/results/demo_TIGER_WOODS_THqn7IjaiEY_1c3e5a96-37ee-4fe4-8df9-1c97cea2cf67.pkl')

# print(df.head())

In [24]:
x = joblib.load('poses/results/demo_TIGER_WOODS_4HzLO88ryCU_6df7f5cc-68ee-40ca-a984-2c3e348767d8.pkl')

#print(x['/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_THqn7IjaiEY_1c3e5a96-37ee-4fe4-8df9-1c97cea2cf67/img/000001.jpg']['smpl'])

In [25]:
joints = x['/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_THqn7IjaiEY_1c3e5a96-37ee-4fe4-8df9-1c97cea2cf67/img/000001.jpg']['3d_joints']

KeyError: '/home/daniel/development/csc2626/final-project/golfdb/data/poses//_DEMO/TIGER_WOODS_THqn7IjaiEY_1c3e5a96-37ee-4fe4-8df9-1c97cea2cf67/img/000001.jpg'

In [26]:
import plotly.graph_objects as go

BONE_CONNECTIONS = {
    'nose_to_neck': [0, 1],
    'neck_to_rshoulder': [1, 2],
    'rshoulder_to_relbow': [2, 3],
    'relbow_to_rwrist': [3, 4],
    'neck_to_lshoulder': [1, 5],
    'lshoulder_to_lelbow': [5, 6],
    'lelbow_to_lwrist': [6, 7],
    'neck_to_midhip': [1, 8],
    'midhip_to_rhip': [8, 9],
    'rhip_to_rknee': [9, 10],
    'rknee_to_rankle': [10, 11],
    'midhip_to_lhip': [8, 12],
    'lhip_to_lknee': [12, 13],
    'lknee_to_lankle': [13, 14],
    'nose_to_reye': [0, 15],
    'nose_to_leye': [0, 16],
    'reye_to_rear': [15, 17],
    'leye_to_lear': [16, 18],
    'lankle_to_lbigtoe': [14, 19],
    'lankle_to_lsmalltoe': [14, 20],
    'lankle_to_lheel': [14, 21],
    'rankle_to_rbigtoe': [11, 22],
    'rankle_to_rsmalltoe': [11, 23],
    'rankle_to_rheel': [11, 24],
}

# Extract the 3D joint coordinates
joints_3d = joints[0][:25]
joints_3d[:, [0, 1, 2]] = joints_3d[:, [0,2,1]]  # Make the 3rd value be the first on the second axis
joints_3d[:, 2] *= -1  # Make the 3rd value negative
#joints_3d[:, 2] = -joints_3d[:, 2]  # Make the 3rd value negative
# Subtract left foot position (left ankle) from all values
left_foot_position = joints_3d[14]  # left ankle joint index is 14
joints_3d -= left_foot_position

# Create a 3D scatter plot

x_coords = [joint[0] for joint in joints_3d]
y_coords = [joint[1] for joint in joints_3d]
z_coords = [joint[2] for joint in joints_3d]
# x_coords = [1, 2, 3]
# y_coords = [4, 5, 6]
# z_coords = [7, 8, 9]

print(x_coords, y_coords, z_coords)
fig = go.Figure(data=[go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers',
    marker=dict(
        size=5,
        color=['blue' if i == 44 else 'red' for i in range(len(joints_3d))],  # Color the 45th joint in blue
    )
)])

# Add lines for each bone defined in BONE_CONNECTIONS
for bone_name, (start_joint, end_joint) in BONE_CONNECTIONS.items():
    fig.add_trace(go.Scatter3d(
        x=[joints_3d[start_joint][0], joints_3d[end_joint][0]],
        y=[joints_3d[start_joint][1], joints_3d[end_joint][1]],
        z=[joints_3d[start_joint][2], joints_3d[end_joint][2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name=bone_name
    ))



# Set labels
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Show the plot
fig.show()


[np.float32(0.36228165), np.float32(0.17967466), np.float32(0.1195876), np.float32(0.114718094), np.float32(0.1759943), np.float32(0.20836842), np.float32(0.1900005), np.float32(0.18386453), np.float32(-0.07640551), np.float32(-0.124421865), np.float32(-0.09256014), np.float32(-0.1657294), np.float32(-0.06443825), np.float32(0.0395559), np.float32(0.0), np.float32(0.3512931), np.float32(0.3727303), np.float32(0.26055205), np.float32(0.3072085), np.float32(0.17847502), np.float32(0.15766636), np.float32(-0.054207664), np.float32(0.002620712), np.float32(-0.07616174), np.float32(-0.22520712)] [np.float32(0.5083344), np.float32(0.4515873), np.float32(0.59264475), np.float32(0.61323106), np.float32(0.55477476), np.float32(0.28295052), np.float32(0.20760264), np.float32(0.3076601), np.float32(0.31762087), np.float32(0.35320807), np.float32(0.40967855), np.float32(0.37313464), np.float32(0.23358613), np.float32(0.13377988), np.float32(0.0), np.float32(0.54341483), np.float32(0.48010138), np.

In [18]:


new_joints_3d = np.array([
    joints_3d[1],  # OP Neck (torso)
    joints_3d[0],  # OP Nose (head)
    joints_3d[8],  # OP MidHip (lower_waist)
    joints_3d[2],  # OP RShoulder (right_upper_arm)
    joints_3d[5],  # OP LShoulder (left_upper_arm)
    joints_3d[8],  # OP MidHip (pelvis)
    joints_3d[3],  # OP RElbow (right_lower_arm)
    joints_3d[6],  # OP LElbow (left_lower_arm)
    joints_3d[9],  # OP RHip (right_thigh)
    joints_3d[12], # OP LHip (left_thigh)
    joints_3d[4],  # OP RWrist (right_hand)
    joints_3d[7],  # OP LWrist (left_hand)
    joints_3d[10], # OP RKnee (right_shin)
    joints_3d[13], # OP LKnee (left_shin)
    (0,0,0),          # Club (not available in JOINT_NAMES)
    joints_3d[11], # OP RAnkle (right_foot)
    joints_3d[14]  # OP LAnkle (left_foot)
])

compute_joint_angles(new_joints_3d)

array([ 1.27593507,  0.38287012,  1.57079633,  1.72135675,  1.57079633,
        1.75307994, -2.32419503,  1.82389793,  1.83986416,  0.04111815,
        0.09699249,  1.17075716,  2.20494371,  1.87297198])

In [9]:
JOINT_NAMES = [
# 25 OpenPose joints (in the order provided by OpenPose)
'OP Nose',
'OP Neck',
'OP RShoulder',
'OP RElbow',
'OP RWrist',
'OP LShoulder',
'OP LElbow',
'OP LWrist',
'OP MidHip',
'OP RHip',
'OP RKnee',
'OP RAnkle',
'OP LHip',
'OP LKnee',
'OP LAnkle',
'OP REye',
'OP LEye',
'OP REar',
'OP LEar',
'OP LBigToe',
'OP LSmallToe',
'OP LHeel',
'OP RBigToe',
'OP RSmallToe',
'OP RHeel',
# 24 Ground Truth joints (superset of joints from different datasets)
'Right Ankle',
'Right Knee',
'Right Hip',
'Left Hip',
'Left Knee',
'Left Ankle',
'Right Wrist',
'Right Elbow',
'Right Shoulder',
'Left Shoulder',
'Left Elbow',
'Left Wrist',
'Neck (LSP)',
'Top of Head (LSP)',
'Pelvis (MPII)',
'Thorax (MPII)',
'Spine (H36M)',
'Jaw (H36M)',
'Head (H36M)',
'ROOT'
]
print(len(JOINT_NAMES))

45


In [6]:
import numpy as np

def compute_all_joint_angles(pose_array):
    """
    Compute all joint angles from OpenPose joint positions to match Isaac Sim format
    
    Args:
        pose_array: numpy array of shape (45, 3) containing joint positions
        
    Returns:
        numpy array of shape (21,) matching initial_positions format
    """
    # OpenPose indices
    NECK_IDX = 1        # OP Neck
    R_SHOULDER_IDX = 2  # OP RShoulder
    R_ELBOW_IDX = 3    # OP RElbow
    R_WRIST_IDX = 4    # OP RWrist
    L_SHOULDER_IDX = 5  # OP LShoulder
    L_ELBOW_IDX = 6    # OP LElbow
    L_WRIST_IDX = 7    # OP LWrist
    MID_HIP_IDX = 8    # OP MidHip
    R_HIP_IDX = 9      # OP RHip
    R_KNEE_IDX = 10    # OP RKnee
    R_ANKLE_IDX = 11   # OP RAnkle
    L_HIP_IDX = 12     # OP LHip
    L_KNEE_IDX = 13    # OP LKnee
    L_ANKLE_IDX = 14   # OP LAnkle
    R_FOOT_IDX = 22    # OP RBigToe
    L_FOOT_IDX = 19    # OP LBigToe
    
    def compute_waist_angles(neck, mid_hip):
        # Calculate waist bend forward/backward and side-to-side
        spine_vector = neck - mid_hip
        spine_vector = spine_vector / np.linalg.norm(spine_vector)
        
        # Forward/backward bend (in sagittal plane)
        sagittal_angle = np.arctan2(spine_vector[2], spine_vector[1])
        
        # Side-to-side bend (in coronal plane)
        coronal_angle = np.arctan2(spine_vector[0], spine_vector[1])
        
        return sagittal_angle / np.pi, coronal_angle / np.pi
    
    def compute_arm_angles(shoulder_idx, elbow_idx, wrist_idx):
        shoulder_pos = pose_array[shoulder_idx]
        elbow_pos = pose_array[elbow_idx]
        wrist_pos = pose_array[wrist_idx]
        
        spine = pose_array[NECK_IDX] - pose_array[MID_HIP_IDX]
        shoulder_line = pose_array[R_SHOULDER_IDX] - pose_array[L_SHOULDER_IDX]
        coronal_normal = np.cross(spine, shoulder_line)
        coronal_normal = coronal_normal / np.linalg.norm(coronal_normal)
        sagittal_normal = np.cross(coronal_normal, spine)
        # Upper arm angles
        upper_arm = elbow_pos - shoulder_pos
        upper_arm = upper_arm / np.linalg.norm(upper_arm)
        
        # Abduction angle relative to coronal plane
        abduction = np.arccos(np.dot(upper_arm, coronal_normal))
        if np.dot(upper_arm, shoulder_line) < 0:  # Check if arm is moving away from body
            abduction = -abduction
            
        # Flexion angle
        flexion = np.arccos(np.dot(upper_arm, sagittal_normal))
        # sagittal_projection = np.array([upper_arm[2], upper_arm[1]])
        # sagittal_projection = sagittal_projection / np.linalg.norm(sagittal_projection)
        # flexion = np.arccos(np.dot(sagittal_projection, [0, 1]))
        # if upper_arm[2] < 0:
        #     flexion = -flexion
            
        # Lower arm angle (elbow)
        forearm = wrist_pos - elbow_pos
        upper_arm = elbow_pos - shoulder_pos
        elbow_angle = np.arccos(np.dot(forearm, upper_arm) / 
                               (np.linalg.norm(forearm) * np.linalg.norm(upper_arm)))
        
        return abduction, flexion, (elbow_angle)
    
    def compute_leg_angles(hip_idx, knee_idx, ankle_idx, foot_idx):
        hip_pos = pose_array[hip_idx]
        knee_pos = pose_array[knee_idx]
        ankle_pos = pose_array[ankle_idx]
        foot_pos = pose_array[foot_idx]
        
        # Thigh angles
        thigh = knee_pos - hip_pos
        thigh = thigh / np.linalg.norm(thigh)
        
        # Calculate three rotation angles for thigh
        thigh_x = np.arctan2(thigh[2], thigh[1])  # forward/backward
        thigh_y = np.arctan2(thigh[0], thigh[1])  # side-to-side
        thigh_z = np.arctan2(thigh[0], thigh[2])  # rotation
        
        # Knee angle
        shank = ankle_pos - knee_pos
        knee_angle = np.arccos(np.dot(thigh, shank) /
                             (np.linalg.norm(thigh) * np.linalg.norm(shank)))
        
        # Foot angles
        foot = foot_pos - ankle_pos
        foot_pitch = np.arctan2(foot[2], foot[1])  # up/down
        foot_roll = np.arctan2(foot[0], foot[1])   # inversion/eversion
        
        return (thigh_x / np.pi, thigh_y / np.pi, thigh_z / np.pi, 
                knee_angle / np.pi, foot_pitch / np.pi, foot_roll / np.pi)
    
    # Calculate all angles
    waist_sagittal, waist_coronal = compute_waist_angles(
        pose_array[NECK_IDX], pose_array[MID_HIP_IDX])
    
    r_abd, r_flex, r_elbow = compute_arm_angles(
        R_SHOULDER_IDX, R_ELBOW_IDX, R_WRIST_IDX)
    l_abd, l_flex, l_elbow = compute_arm_angles(
        L_SHOULDER_IDX, L_ELBOW_IDX, L_WRIST_IDX)
    
    pelvis_angle = np.arctan2(
        pose_array[R_HIP_IDX][0] - pose_array[L_HIP_IDX][0],
        pose_array[R_HIP_IDX][2] - pose_array[L_HIP_IDX][2]) / np.pi
    
    r_thigh_x, r_thigh_y, r_thigh_z, r_knee, r_foot_pitch, r_foot_roll = compute_leg_angles(
        R_HIP_IDX, R_KNEE_IDX, R_ANKLE_IDX, R_FOOT_IDX)
    l_thigh_x, l_thigh_y, l_thigh_z, l_knee, l_foot_pitch, l_foot_roll = compute_leg_angles(
        L_HIP_IDX, L_KNEE_IDX, L_ANKLE_IDX, L_FOOT_IDX)
    
    # Assemble in the same order as initial_positions
    return np.array([
        waist_coronal,  # lower_waist
        waist_sagittal,   # lower_waist
        r_abd,          # right_upper_arm
        r_flex,         # right_upper_arm
        l_abd,          # left_upper_arm
        l_flex,         # left_upper_arm
        pelvis_angle,   # pelvis
        r_elbow,        # right_lower_arm
        l_elbow,        # left_lower_arm
        r_thigh_x,      # right_thigh: x
        r_thigh_y,      # right_thigh: y
        r_thigh_z,      # right_thigh: z
        l_thigh_x,      # left_thigh: x
        l_thigh_y,      # left_thigh: y
        l_thigh_z,      # left_thigh: z
        r_knee,         # right_knee
        l_knee,         # left_knee
        r_foot_pitch,   # right_foot
        r_foot_roll,    # right_foot
        l_foot_pitch,   # left_foot
        l_foot_roll,    # left_foot
    ])

# Example usage:
# pose_data = np.random.rand(45, 3)  # Your pose data here
# joint_angles = compute_all_joint_angles(pose_data)

In [7]:
joint_angles = compute_all_joint_angles(joints_3d)
print(joint_angles)

[ 0.8143025  -0.89417243  0.9508878   1.4707422  -1.0989401   1.6311872
 -0.85204905  0.40094122  0.6913518  -0.04598082  0.0260668   0.836486
  0.08649698  0.08993983  0.25654003  0.11140303  0.11913639 -0.22328617
  0.34219608 -0.16721205  0.3553766 ]


In [7]:
import numpy as np
isaac_joint_ex = np.array([[[-6.3307e-02,  4.1133e-02,  1.0862e+00,  8.2588e-01,  4.2306e-02,
           5.6173e-01, -2.4218e-02, -3.2339e-01,  1.3146e-01, -1.0134e-01,
           1.8591e-01,  5.0460e-01, -2.2101e-02],
         [ 1.1259e-01,  2.2687e-02,  1.1556e+00,  8.2588e-01,  4.2306e-02,
           5.6173e-01, -2.4218e-02, -2.8213e-01,  1.1121e-01, -2.1674e-01,
           1.8591e-01,  5.0460e-01, -2.2101e-02],
         [-2.5426e-01,  6.1176e-02,  9.5980e-01,  9.9602e-01,  1.2835e-02,
           5.0397e-02,  7.2358e-02, -3.5577e-01,  1.5930e-01, -2.6902e-02,
           2.4795e-01,  1.2763e-01,  4.1328e-02],
         [-2.2640e-02, -1.3388e-01,  1.1008e+00,  8.2717e-01,  3.5315e-01,
           4.3171e-01,  6.8515e-02, -4.1392e-01,  1.5754e-01, -1.5356e-01,
           2.4475e-01,  7.4676e-01, -4.8113e-03],
         [ 7.1208e-03,  2.0450e-01,  1.1153e+00,  8.1802e-01, -1.3191e-01,
           5.5824e-01, -4.2620e-02, -2.5521e-01,  1.2960e-01, -1.4447e-01,
          -2.5006e-02, -2.7023e-01, -2.3035e-01],
         [-2.6974e-01,  5.4893e-02,  7.9583e-01,  9.9563e-01, -3.3860e-02,
           4.4935e-02,  7.4529e-02, -3.6865e-01,  1.4968e-01, -2.1715e-02,
          -2.3476e-01,  5.6380e-02,  8.7674e-02],
         [-8.3173e-02, -9.7493e-02,  7.9715e-01,  6.6653e-01,  1.9141e-01,
           7.1758e-01, -6.4605e-02, -6.0180e-01,  2.1870e-01, -1.9162e-01,
           2.4307e-01,  7.5047e-01, -4.0323e-03],
         [-1.0610e-01,  3.0824e-01,  8.4400e-01,  6.5039e-01,  1.6765e-02,
           7.4971e-01,  1.2101e-01, -1.8250e-01,  1.1743e-01, -1.3805e-01,
          -2.5836e-02, -2.7134e-01, -2.3043e-01],
         [-2.5797e-01, -4.6732e-02,  7.6215e-01,  9.6755e-01, -1.0398e-01,
          -2.1086e-01,  9.2655e-02, -3.2300e-01,  1.2735e-01,  2.0599e-02,
          -1.7400e-01, -2.3558e-01,  1.3588e-01],
         [-2.8826e-01,  1.5059e-01,  7.5001e-01,  9.5321e-01,  8.0019e-03,
          -2.8794e-01,  9.1777e-02, -3.9120e-01,  8.7349e-02, -5.4783e-02,
          -2.6924e-01,  8.5775e-02, -1.3949e-01],
         [ 1.4263e-01,  3.9141e-02,  6.3119e-01,  6.6653e-01,  1.9141e-01,
           7.1758e-01, -6.4605e-02, -6.6380e-01,  2.3842e-01, -2.5975e-01,
           2.4307e-01,  7.5047e-01, -4.0323e-03],
         [ 6.6365e-02,  1.9521e-01,  6.1016e-01,  6.5039e-01,  1.6765e-02,
           7.4971e-01,  1.2101e-01, -1.6380e-01,  9.4535e-02, -1.1319e-01,
          -2.5836e-02, -2.7134e-01, -2.3043e-01],
         [-9.9379e-02, -1.0447e-01,  4.0015e-01,  9.8507e-01, -1.2768e-01,
           1.0125e-01,  5.5628e-02, -1.4282e-01,  8.1667e-02,  2.8256e-02,
          -1.0259e-01, -7.4234e-01,  2.6256e-01],
         [-7.2986e-02,  1.6679e-01,  4.1131e-01,  9.9121e-01, -1.0198e-02,
          -9.4919e-02,  9.1559e-02, -2.0629e-01,  1.4250e-02, -2.8939e-02,
          -3.5942e-02, -1.1914e+00, -9.0643e-02],
         [ 8.5325e-02, -4.5011e-02,  9.3270e-01,  6.5039e-01,  1.6765e-02,
           7.4971e-01,  1.2101e-01, -4.6299e-02, -1.6010e-02,  3.8042e-03,
          -2.5836e-02, -2.7134e-01, -2.3043e-01],
         [-1.5227e-01, -2.0182e-01,  2.8334e-02,  9.9016e-01, -1.0201e-01,
          -1.9005e-02,  9.3868e-02, -4.7292e-03,  1.7406e-02,  2.8634e-02,
          -5.2376e-01, -2.0127e-01, -3.3221e-02],
         [-1.4144e-02,  1.6601e-01,  2.6956e-02,  9.9554e-01,  6.6397e-04,
           3.7092e-04,  9.4339e-02, -5.3240e-03,  7.7487e-04,  1.1875e-02,
          -6.1000e-02, -1.7204e-01, -5.9444e-03]]])

In [8]:
isaac_joints_3d = isaac_joint_ex[0, :, :3]
isaac_joints_3d.shape

(17, 3)

In [15]:
import plotly.graph_objects as go

BONE_CONNECTIONS = {
    'torso_to_head': [0, 1],
    'torso_to_lower_waist': [0, 2],
    'torso_to_right_upper_arm': [0, 3],
    'torso_to_left_upper_arm': [0, 4],
    'lower_waist_to_pelvis': [2, 5],
    'right_upper_arm_to_right_lower_arm': [3, 6],
    'left_upper_arm_to_left_lower_arm': [4, 7],
    'right_lower_arm_to_right_hand': [6, 10],
    'left_lower_arm_to_left_hand': [7, 11],
    'pelvis_to_right_thigh': [5, 8],
    'pelvis_to_left_thigh': [5, 9],
    'right_thigh_to_right_shin': [8, 12],
    'left_thigh_to_left_shin': [9, 13],
    'right_shin_to_right_foot': [12, 15],
    'left_shin_to_left_foot': [13, 16],  # Assuming Club is held in the right hand
}

# Extract the 3D joint coordinates
print(isaac_joints_3d)
joints_3d = isaac_joints_3d  # joints[0][:25]

# Create a 3D scatter plot
# print(joints_3d)
# print(len(joints_3d))
x_coords = [joint[0] for joint in joints_3d]
y_coords = [joint[1] for joint in joints_3d]
z_coords = [joint[2] for joint in joints_3d]
# x_coords = [1, 2, 3]
# y_coords = [4, 5, 6]
# z_coords = [7, 8, 9]

print(x_coords, y_coords, z_coords)
fig = go.Figure(data=[go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers',
    marker=dict(
        size=5,
        color=['blue' if i == 0 else 'red' for i in range(len(joints_3d))],  # Color the 45th joint in blue
    )
)])

# # Add lines for each bone defined in BONE_CONNECTIONS
for bone_name, (start_joint, end_joint) in BONE_CONNECTIONS.items():
    fig.add_trace(go.Scatter3d(
        x=[joints_3d[start_joint][0], joints_3d[end_joint][0]],
        y=[joints_3d[start_joint][1], joints_3d[end_joint][1]],
        z=[joints_3d[start_joint][2], joints_3d[end_joint][2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name=bone_name
    ))



# Set labels
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Show the plot
fig.show()


[[-0.063307   0.041133   1.0862   ]
 [ 0.11259    0.022687   1.1556   ]
 [-0.25426    0.061176   0.9598   ]
 [-0.02264   -0.13388    1.1008   ]
 [ 0.0071208  0.2045     1.1153   ]
 [-0.26974    0.054893   0.79583  ]
 [-0.083173  -0.097493   0.79715  ]
 [-0.1061     0.30824    0.844    ]
 [-0.25797   -0.046732   0.76215  ]
 [-0.28826    0.15059    0.75001  ]
 [ 0.14263    0.039141   0.63119  ]
 [ 0.066365   0.19521    0.61016  ]
 [-0.099379  -0.10447    0.40015  ]
 [-0.072986   0.16679    0.41131  ]
 [ 0.085325  -0.045011   0.9327   ]
 [-0.15227   -0.20182    0.028334 ]
 [-0.014144   0.16601    0.026956 ]]
[np.float64(-0.063307), np.float64(0.11259), np.float64(-0.25426), np.float64(-0.02264), np.float64(0.0071208), np.float64(-0.26974), np.float64(-0.083173), np.float64(-0.1061), np.float64(-0.25797), np.float64(-0.28826), np.float64(0.14263), np.float64(0.066365), np.float64(-0.099379), np.float64(-0.072986), np.float64(0.085325), np.float64(-0.15227), np.float64(-0.014144)] [np.float

In [10]:
isaac_joint_names = ['lower_waist', 'lower_waist', 'right_upper_arm', 'left_upper_arm', 'pelvis', 'right_lower_arm', 'left_lower_arm', 'right_thigh', 'left_thigh', 'right_shin', 'left_shin', 'right_foot', 'left_foot']


In [11]:
len(isaac_joint_names)

13

In [19]:
def get_waist_angles(neck, mid_hip, base, left_knee, right_knee):
    spine = neck - mid_hip
    vertical = base - mid_hip
    vertical = base_spine / np.linalg.norm(base_spine)
    horizontal = left_knee - right_knee
    horizontal = horizontal / np.linalg.norm(horizontal)

    # Normalize the length of the spine vector
    spine = spine / np.linalg.norm(spine)
    
    # Forward/backward bend (in sagittal plane)

    # Opposite over adjacent (z over y)
    forward_angle = np.arccos(np.dot(spine, vertical))
    
    side_angle = np.arccos(np.arctan2(spine, horizontal))
    
    return forward_angle, side_angle

def get_arm_angles(shoulder, elbow, wrist, neck, mid_hip):
        # Shoulder angle is dependent on the rotation of the spine and hips

        # Here we formulate shoulder angle in terms of abduction 
        # (moving away from sagital plane) and flexion (moving away from coronal plane)
        spine = neck - mid_hip
        shoulder_line = shoulder - elbow
        coronal_normal = np.cross(spine, shoulder_line)
        coronal_normal = coronal_normal / np.linalg.norm(coronal_normal)
        sagittal_normal = np.cross(coronal_normal, spine)

        # Shoulder angles
        upper_arm = elbow - shoulder
        upper_arm = upper_arm / np.linalg.norm(upper_arm)
        
        
        shoulder_abduction = np.arccos(np.dot(upper_arm, coronal_normal))
        shoulder_flexion = np.arccos(np.dot(upper_arm, sagittal_normal))

        # Elbow angle
        forearm = wrist - elbow
        forearm = forearm / np.linalg.norm(forearm)
        upper_arm = elbow - shoulder
        upper_arm = upper_arm / np.linalg.norm(upper_arm)
        elbow_angle = np.arccos(np.dot(forearm, upper_arm))
        
        return shoulder_abduction, shoulder_flexion, elbow_angle

def get_leg_angles(hip, knee, ankle, foot):
        # Thigh angles
        thigh = knee - hip
        thigh = thigh / np.linalg.norm(thigh)
        
        # Hip angles are simply the angle on each 3D rotation axis
        thigh_forward = np.arctan2(thigh[2], thigh[1])
        thigh_out = np.arctan2(thigh[0], thigh[1]) 
        thigh_twist = np.arctan2(thigh[0], thigh[2])  
        
        # Knee angle is the angle between the thigh and shin
        shin = ankle - knee
        shin = shin / np.linalg.norm(shin)
        knee_angle = np.arccos(np.dot(thigh, shin))
        
        # Foot tracking is not accurate enough, so we don't compute foot angles
        
        return thigh_forward, thigh_out, thigh_twist, knee_angle
    

def compute_joint_angles(pose_array):
    '''
    Assume pose_array is in the following format:

    ['torso', 'head', 'lower_waist', 'right_upper_arm', 'left_upper_arm', 
    'pelvis', 'right_lower_arm', 'left_lower_arm', 'right_thigh', 
    'left_thigh', 'right_hand', 'left_hand', 'right_shin', 'left_shin', 
    'Club', 'right_foot', 'left_foot']
    '''

    # Center the pose at the left foot
    pose_array = pose_array - pose_array[16]

    
    waist_forwards, waist_sideways = get_waist_angles(pose_array[1], pose_array[0], (pose_array[16] + pose_array[15]) / 2, pose_array[12], pose_array[13])
    right_shoulder_abduction, right_shoulder_flexion, right_elbow_angle = get_arm_angles(pose_array[3], pose_array[6], pose_array[10], pose_array[1], pose_array[0])
    left_shoulder_abduction, left_shoulder_flexion, left_elbow_angle = get_arm_angles(pose_array[4], pose_array[7], pose_array[11], pose_array[1], pose_array[0])
    right_thigh_forward, right_thigh_out, right_thigh_twist, right_knee_angle = get_leg_angles(pose_array[8], pose_array[12], pose_array[13], pose_array[15])
    left_thigh_forward, left_thigh_out, left_thigh_twist, left_knee_angle = get_leg_angles(pose_array[9], pose_array[13], pose_array[14], pose_array[16])

    return np.array([
        waist_forwards, 
        waist_sideways, 
        right_shoulder_abduction, 
        right_shoulder_flexion, 
        left_shoulder_abduction, 
        left_shoulder_flexion, 
        right_thigh_forward, 
        right_thigh_out, 
        right_thigh_twist, 
        left_thigh_forward, 
        left_thigh_out, 
        left_thigh_twist, 
        right_knee_angle, 
        left_knee_angle
        ])

isaac_joint_angles = compute_joint_angles(isaac_joints_3d)
print(isaac_joint_angles)

[ 1.83058242  1.67528261  1.57079633  1.7305368   1.57079633  1.70939749
 -1.72896134  1.91994901  2.7286815  -1.52300281  1.49568497  2.57541498
  1.71370235  2.2415169 ]


In [ ]:
array([ 1.27593507,  0.38287012,  1.57079633,  1.72135675,  1.57079633,
        1.75307994, -2.32419503,  1.82389793,  1.83986416,  0.04111815,
        0.09699249,  1.17075716,  2.20494371,  1.87297198])